<a href="https://colab.research.google.com/github/Jakelinecs/Tareas-Machine-Learning/blob/main/N27.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Reescritura del modelo de regresión de House Prices usando la API de Keras.
Objetivo: Predecir el precio de venta (SalePrice) usando GrLivArea y YearBuilt.
Keras se utiliza a través del módulo tf.keras.
"""
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# --- CONFIGURACIÓN DE DATOS ---

# Carga del conjunto de datos (asumiendo que train.csv está en la misma carpeta)
try:
    df = pd.read_csv("train.csv")
except FileNotFoundError:
    print("Error: 'train.csv' no encontrado. Asegúrese de que el archivo existe.")
    # Usar datos dummy para demostración si el archivo falta
    data = {
        'GrLivArea': np.random.randint(1000, 3000, 1460),
        'YearBuilt': np.random.randint(1900, 2010, 1460),
        'SalePrice': np.random.randint(100000, 400000, 1460)
    }
    df = pd.DataFrame(data)

# Variables
y_raw = df["SalePrice"]
X_raw = df.loc[:, ["GrLivArea", "YearBuilt"]]

# Preprocesamiento
# 1. Transformación logarítmica de la variable objetivo (SalePrice), común en regresión de precios
y_log = np.log1p(y_raw)
y = np.array(y_log)[:, np.newaxis] # Forma (n_samples, 1)

# 2. Estandarización de las características
scaler_X = StandardScaler()
X = scaler_X.fit_transform(X_raw)

# División en train, test y val
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

# Hiperparámetros
batch_size = 32
num_epochs = 200

n_hidden1 = 100
n_hidden2 = 50
n_input = X_train.shape[1]
n_output = 1 # Salida única para regresión (SalePrice)

# Establecer la semilla para reproducibilidad
tf.random.set_seed(0)

# --- DEFINICIÓN DEL MODELO KERAS (API Secuencial) ---
print("--- Creando modelo Keras para Regresión ---")

# 1. Definición de la estructura del modelo
model = Sequential([
    # Capa oculta 1: n_hidden1 unidades, activación ReLU
    Dense(n_hidden1, activation='relu', input_shape=(n_input,)),
    # Capa oculta 2: n_hidden2 unidades, activación ReLU
    Dense(n_hidden2, activation='relu'),
    # Capa de salida: 1 unidad.
    # IMPORTANTE: NO se usa función de activación (salida lineal) para regresión
    Dense(n_output, activation='linear')
])

# 2. Compilación del modelo (equivalente a loss_op y optimizer.minimize en TF1.x)
# loss: 'mse' (Mean Squared Error) es la pérdida estándar para regresión
# metrics: 'RootMeanSquaredError' (RMSE) es más interpretable que MSE
model.compile(optimizer='adam',
              loss='mse',
              metrics=[tf.keras.metrics.RootMeanSquaredError()])

model.summary()

# --- ENTRENAMIENTO Y EVALUACIÓN ---

print("\n--- Regresión (House Prices) con Keras ---")

# 3. Entrenamiento del modelo
history = model.fit(
    X_train, y_train,
    batch_size=batch_size,
    epochs=num_epochs,
    validation_data=(X_val, y_val),
    verbose=2 # Muestra una línea por época
)

# 4. Evaluación del modelo con los datos de prueba
print("\n--- Evaluación del modelo en datos de prueba ---")
# Devuelve la pérdida (MSE) y la métrica (RMSE)
loss, test_rmse = model.evaluate(X_test, y_test, verbose=0)

print("--- Entrenamiento Completado ---")
# El RMSE se calcula sobre el log(1+SalePrice)
print("RMSE de prueba (escala log SalePrice) : {:.4f}".format(test_rmse))



Error: 'train.csv' no encontrado. Asegúrese de que el archivo existe.
--- Creando modelo Keras para Regresión ---


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 100)            │           300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 50)             │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,401 (21.10 KB)

 Trainable params: 5,401 (21.10 KB)

 Non-trainable params: 0 (0.00 B)


--- Regresión (House Prices) con Keras ---
Epoch 1/200
30/30 - 3s - 103ms/step - loss: 135.4852 - root_mean_squared_error: 11.6398 - val_loss: 111.2398 - val_root_mean_squared_error: 10.5470
Epoch 2/200
30/30 - 0s - 4ms/step - loss: 83.2142 - root_mean_squared_error: 9.1222 - val_loss: 45.8917 - val_root_mean_squared_error: 6.7743
Epoch 3/200
30/30 - 0s - 5ms/step - loss: 23.9640 - root_mean_squared_error: 4.8953 - val_loss: 9.1543 - val_root_mean_squared_error: 3.0256
Epoch 4/200
30/30 - 0s - 4ms/step - loss: 7.4670 - root_mean_squared_error: 2.7326 - val_loss: 6.4171 - val_root_mean_squared_error: 2.5332
Epoch 5/200
30/30 - 0s - 4ms/step - loss: 5.9650 - root_mean_squared_error: 2.4423 - val_loss: 5.2829 - val_root_mean_squared_error: 2.2984
Epoch 6/200
30/30 - 0s - 5ms/step - loss: 4.9943 - root_mean_squared_error: 2.2348 - val_loss: 4.3552 - val_root_mean_squared_error: 2.0869
Epoch 7/200
30/30 - 0s - 4ms/step - loss: 4.1284 - root_mean_squared_error: 2.0318 - val_loss: 3.5302 - v